In [1]:
import tarfile
import os
import json
import boto3
import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role, Session
from sagemaker.serializers import IdentitySerializer
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
with tarfile.open('model.tar.gz', 'w:gz') as f:
    f.add('model.pt')
f.close()

In [3]:
sess = Session()

role = get_execution_role()

%store -r pt_malconv_model_data

try:
    pt_malconv_model_data
except NameError:
    import json

    # copy a pretrained model from a public public to your default bucket
    s3 = boto3.client("s3")
    
    # upload to default bucket
    pt_malconv_model_data = sess.upload_data(
        path="model.tar.gz", bucket=sess.default_bucket(), key_prefix="model/pytorch"
    )

no stored variable or alias pt_malconv_model_data


In [4]:
print(pt_malconv_model_data)

s3://sagemaker-us-east-1-975050042186/model/pytorch/model.tar.gz


In [5]:
model = PyTorchModel(
    entry_point="inference.py",
    role=role,
    model_data=pt_malconv_model_data,
    framework_version="1.5.0",
    py_version="py3",
)

In [6]:
instance_type = "ml.m5.xlarge"

predictor = model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    #serializer=JSONSerializer(),
    serializer=IdentitySerializer(content_type='application/octet-stream'),  # Updated
    deserializer=JSONDeserializer(),
)

------!